# Face detection and embeddings collecting
This is one of test tasks for some company, where I should make a system for face detecting and collecting embeddings, based on dlib instruments. 




## Preparatory part

### Import (or install and import) necessary libs

In [0]:
DEBUG = True

In [0]:
%matplotlib inline

import bz2
import cv2
import os
import requests
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib
import matplotlib.pyplot as plt
import keras
from keras.models import load_model 

try:
    import dlib
    if DEBUG:
        print("All libs are installed")
except:
    if DEBUG:
        print("Installing the required libraries")
  
    # dlib installation
    !apt update
    !apt install -y cmake
    !pip install dlib
    import dlib


Using TensorFlow backend.


Installing the required libraries
Hit:1 http://security.ubuntu.com/ubuntu artful-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu artful InRelease
Hit:3 http://archive.ubuntu.com/ubuntu artful-updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu artful-backports InRelease
Reading package lists... Done
Building dependency tree       
Reading state information... Done
All packages are up to date.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  cmake-data libarchive13 libjsoncpp1 liblzo2-2 librhash0 libuv1
Suggested packages:
  ninja-build lrzip
The following NEW packages will be installed:
  cmake cmake-data libarchive13 libjsoncpp1 liblzo2-2 librhash0 libuv1
0 upgraded, 7 newly installed, 0 to remove and 0 not upgraded.
Need to get 4,930 kB of archives.
After this operation, 25.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu artful/main a

### Check existing of weights files in cloud dir. If it's not exist, download and unpack

In [0]:
face_detector_weights = os.path.isfile("mmod_human_face_detector.dat.bz2")
facenet_model         = os.path.isfile("facenet_keras.h5")

if face_detector_weights and facenet_model:
    if DEBUG:
        print("Model exist")
else:
    if DEBUG:
        print("Downloading weights")
    # Download pretrained cnn_face_detection_model_v1 from dlib and facenet
    !wget "http://dlib.net/files/mmod_human_face_detector.dat.bz2"
    !wget -O facenet_keras.h5 "https://s185f.storage.yandex.net/rdisk/594789e50af533c4dae873280d755d113dad36c805811b890b42b8c7adb395cb/5bb1a171/xiCnyZq0qx6g-h6yIY-VdMjxvJjJ7MyF6tI5sedijBh4gG6evns74XjMPwBKjqmpUhXcwQ66NZ_XEP2PLLH1mw==?uid=437831410&filename=facenet_keras.h5&disposition=attachment&hash=&limit=0&content_type=application%2Fx-hdf&fsize=92397640&hid=75f38b2414140fcc52f40ca5659a7462&media_type=data&tknv=v2&etag=d4169b76ead0a7a58c5ba7ca4c0b505b&rtoken=a3jIiqUtznsb&force_default=yes&ycrid=na-d9b081e66c7333a7855196900263f819-downloader10e&ts=577232da67e40&s=4e5d3092ea0a89eb38107b34e8fa2ffd1e7b2764035b94cd052581aacac56a48&pb=U2FsdGVkX1_nCthRxfzMl_utdUzktNGlJPzh-IjhWru80fQknF9xrqx3DyGGs7Ji5_4RtTRGBFzcDcH9ZZAyRCRddZTGNa_OxLTqBf1nmMU"

# Unpack and save cnn face detector from dlib
!bzip2 -dk "mmod_human_face_detector.dat.bz2" 



Redirecting output to ‘wget-log’.

Redirecting output to ‘wget-log.1’.


In [0]:
# Optional part for garbage collection in notebook dirrectory

# !rm fa* mm* wg*
!ls

facenet_keras.h5	      mmod_human_face_detector.dat.bz2	wget-log
mmod_human_face_detector.dat  sample_data			wget-log.1


#### Data standartization from FaceNet predict
Code of function from https://github.com/nyoki-mtl/keras-facenet/blob/master/notebook/demo-images.ipynb 

In [0]:
def prewhiten(x):
    if x.ndim == 4:
        axis = (1, 2, 3)
        size = x[0].size
    elif x.ndim == 3:
        axis = (0, 1, 2)
        size = x.size
    else:
        raise ValueError('Dimension should be 3 or 4')

    mean = np.mean(x, axis=axis, keepdims=True)
    std = np.std(x, axis=axis, keepdims=True)
    std_adj = np.maximum(std, 1.0/np.sqrt(size))
    y = (x - mean) / std_adj
    return y

## Image downlader

In [0]:
def image_downloader():
    """
    This function allows you to download n images with using its url.
    To complete adding photos just press enter with empty input
    
    :return: list of images as numpy arrays
    """
    
    imgs = []
    loop = True
    while loop:
        url = input("Type url here: ")

        if DEBUG and not url:
            url  = "https://cs.pikabu.ru/post_img/big/2013/12/16/7/1387185457_631216170.jpg"
#             url  = "http://www.fotovam.ru/para/kalug/1.jpg"
            loop = False
            
        if not url:
            loop = False
            break
        img = Image.open(requests.get(url, stream=True).raw)
        img = np.asarray(img)
        imgs.append(img)

    if DEBUG:
        plt.figure(figsize=(20,10))
        columns = 5
        for i, image in enumerate(imgs):
            plt.subplot(len(imgs) / columns + 1, columns, i + 1)
            plt.grid(False)
            plt.imshow(image)
    print(" You download {} photos".format(len(imgs)))
    
    if len(imgs) == 0: ValueError: print("Sorry. There are no photos for detecting. It's unnecessary to execute code.")
    
    return imgs

## Class for face detection system

In [0]:
class FaceDetection():
    
    if DEBUG:
        def __init__(self, det, nn):
        # Initialize face detector and FaceNet
            self.cnn_dlib_face_detector = det
            self.face_net = nn     
    else:
        def __init(self):
            self.cnn_dlib_face_detector = dlib.cnn_face_detection_model_v1('mmod_human_face_detector.dat')
            self.face_net = load_model("facenet_keras.h5")
        
    def _get_faces(self, img, size):
        """
        :param img: (numpy array) take image (w,h,ch)
        :param size: (int) number of pixels (width, height) used in cropping
        :return: bounding boxes of faces in the photo
        """
        faces_coord = self.cnn_dlib_face_detector(img, 1)
        
        if DEBUG:
            crop_img = []
        # loop over detected faces
        faces = []
        for face in faces_coord:
            x = face.rect.left()
            y = face.rect.top()
            w = face.rect.right() - x
            h = face.rect.bottom() - y
            margin = 10
            cropped = img[y-margin//2:y+h+margin//2, x-margin//2:x+w+margin//2, :]
            aligned = Image.fromarray(np.uint8(cropped)).resize((160,160))

            # normalize and append
            faces.append(prewhiten(np.asarray(aligned)))

            if DEBUG: 
                cv2.rectangle(img, (x, y), (x + w, y + h), (0, 0, 255), 3)
                crop_img.append(cropped)
        if DEBUG:
            columns = 5
            plt.figure(figsize=(10, 5))
            for i, image in enumerate(crop_img):
                plt.subplot(len(crop_img) / columns + 1, columns, i + 1)
                plt.grid(False)
                plt.imshow(image)
        
            plt.figure(figsize=(10, 5))
            plt.grid(False)
            plt.imshow(img)

        return faces
     
    def get_embeddings(self, img, size=160):
        """
        :param img: (numpy array) take image (w,h,ch)
        :param size: (int) number of pixels (width, height) used in cropping 
        :return: (numpy array) embedding from FaceNet, (int) number of faces found in the photo 
        """
        faces = self._get_faces(img, size)
        embeddings = []
        for face in faces:
            face = face.reshape((1, size, size, 3))
            emb = self.face_net.predict(face)
            embeddings.append(emb)

        return embeddings


## Practical part
- Initialize detector (with CNN for face detection and FaceNet for embeddings)
- Download images from its urls
- Find faces in each photo and append embeddings

In [0]:
if DEBUG:
    cnn_dlib_face_detector = dlib.cnn_face_detection_model_v1('mmod_human_face_detector.dat')
    face_net = load_model("facenet_keras.h5")

/usr/local/lib/python3.6/dist-packages/keras/models.py:282: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [0]:
if DEBUG:
    detector = FaceDetection(cnn_dlib_face_detector, face_net)

else:
    detector = FaceDetection()

In [0]:
size = 160
imgs = image_downloader()

Type url here: http://www.fotovam.ru/para/kalug/1.jpg
Type url here: 
 You download 1 photos


In [0]:
emb = []
for idx, img in enumerate(imgs):
    embeddings = detector.get_embeddings(img, size)
    emb.append(embeddings)
    print("On {} photo {} faces founded".format(idx+1, len(emb[idx])))

On 1 photo 9 faces founded
